<a href="https://colab.research.google.com/github/inesdarosa/titulares_diarios/blob/master/diario_geral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
 
files.upload()

In [ ]:
!pip install pysentimiento

In [ ]:
import pandas as pd

In [ ]:
base_tit_gral = pdpd.read_csv('base_titulares.csv', header=None, encoding='iso8859_2')# acá debo 

In [ ]:
base_tit_gral.columns = ['titular', 'fecha', 'diario']
base_tit_gral.head(10)

In [ ]:
base_tit_gral

,titular,fecha,diario
0,"""Mirá la ceremonia de inauguración de los Jueg...","""2021-07-23""",elobse
1,"""El contrasentido del clásico: Peńarol perdió ...","""2021-07-23""",elobse
2,"""Los sueldos de $ 200.000 en el Directorio de ...","""2021-07-23""",elobse
3,"""żDónde se podrán ver los Juegos Olímpicos de ...","""2021-07-23""",elobse
4,"""Crece interés de argentinos por Montevideo: c...","""2021-07-23""",elobse
...,...,...,...
96926,"""La Unión Económica Euroasiática en el context...","""2022-07-19""",republ
96927,"""La fuerza de las ideas batllistas en la educa...","""2022-07-19""",republ
96928,"""Suplemento Ciencia & Tecnología  19 de Julio...","""2022-07-19""",republ
96929,"""Ghana declara un brote tras detectar dos muer...","""2022-07-19""",republ


In [ ]:
from pysentimiento import create_analyzer
analyzer = create_analyzer(task="sentiment", lang="es")

Downloading:   0%|          | 0.00/925 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/415M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/334 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/838k [00:00<?, ?B/s]

In [ ]:
lista_titulares = base_tit_gral['titular'].tolist()
lista_titulares[0:4]

['"Mirá la ceremonia de inauguración de los Juegos Olímpicos Tokio 2020"            ',
 '"El contrasentido del clásico: Peńarol perdió en su cancha, pero ganó sin discusión"          ',
 '"Los sueldos de $ 200.000 en el Directorio de la Caja de Profesionales y la propuesta para que sean honorarios"   ',
 '"żDónde se podrán ver los Juegos Olímpicos de Tokio 2020 en Uruguay?"           ']

In [ ]:
from tqdm.auto import tqdm

In [ ]:
'''Esto demoró mucho y se cortó, capaz no es la mejor opción.
sentimiento_0 = map(lambda x: analyzer.predict(x).output, lista_titulares)
sentimiento = list(sentimiento_0)'''

In [ ]:
'''Esta funciona pero tambien es lenta'''

sentimiento = []
for i in tqdm(lista_titulares):
  sentimiento.append(analyzer.predict(i).output)

  0%|          | 0/96931 [00:00<?, ?it/s]

In [ ]:
'''Por ahora lo dejo un ratito, pero funciona
sentimiento = []
for i in range(96931):
  sentimiento.append(analyzer.predict(base_tit_gral.iloc[i,0]).output)'''

In [ ]:
base_tit_gral['sentimiento'] = sentimiento
base_tit_gral.head(10)

,titular,fecha,diario,sentimiento
0,"""Mirá la ceremonia de inauguración de los Jueg...","""2021-07-23""",elobse,NEU
1,"""El contrasentido del clásico: Peńarol perdió ...","""2021-07-23""",elobse,NEU
2,"""Los sueldos de $ 200.000 en el Directorio de ...","""2021-07-23""",elobse,NEU
3,"""żDónde se podrán ver los Juegos Olímpicos de ...","""2021-07-23""",elobse,NEU
4,"""Crece interés de argentinos por Montevideo: c...","""2021-07-23""",elobse,POS
5,"""Delgado y Orsi, la tensión entre dos referent...","""2021-07-23""",elobse,NEU
6,"""El antiguo PES será gratuito: żhay chances de...","""2021-07-23""",elobse,NEU
7,"""El correctivo salarial que se debe pagar a tr...","""2021-07-23""",elobse,NEU
8,"""\""Obras que no se harán\"", \""sorpresa\"" y \""f...","""2021-07-23""",elobse,NEG
9,"""El dilema de los patrocinadores japoneses ant...","""2021-07-23""",elobse,NEG


In [ ]:
# Analicemos la cantidad de datos positivos, negativos y neutros que tenemos. Hay algo mal, me cuenta cosas que no se si están bien.
#print(base_tit_gral[ base_tit_gral['sentimiento'] == 'POS'].size)
#print(base_tit_gral[ base_tit_gral['sentimiento'] == 'NEG'].size)
#print(base_tit_gral[ base_tit_gral['sentimiento'] == 'NEU'].size)

In [ ]:
# Esto me da la cantidad de titulares de cada etiqueta

freq = base_tit_gral.groupby(['sentimiento']).count() 
print(freq)

             titular  fecha  diario
sentimiento                        
NEG            14573  14573   14573
NEU            74560  74560   74560
POS             7798   7798    7798


In [ ]:
#Descartar los sentimientos del tipo 'Neutral'
base_tit_gral = base_tit_gral[base_tit_gral.sentimiento != "NEU"]

In [ ]:
base_tit_gral

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
max_features = 2000
#Inicializo el Tokenizer
tokenizer = Tokenizer(num_words=max_features, split=' ')
#Entrenamiento
tokenizer.fit_on_texts(base_tit_gral['titular'].values)
#texts_to_sequences transforma el input en arrays numéricos
X = tokenizer.texts_to_sequences(base_tit_gral['titular'].values)
#pads_sequences transforma las secuencias al mismo largo
X = pad_sequences(X)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout

In [ ]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
model.add(Dropout(0.5))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 30, 128)           256000    
                                                                 
 dropout (Dropout)           (None, 30, 128)           0         
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 2)                 394       
                                                                 
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
Y = pd.get_dummies(base_tit_gral['sentimiento']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(14988, 30) (14988, 2)
(7383, 30) (7383, 2)


In [ ]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 10, batch_size=batch_size, verbose = 2)

Epoch 1/10
469/469 - 91s - loss: 0.3475 - accuracy: 0.8403 - 91s/epoch - 195ms/step
Epoch 2/10
469/469 - 93s - loss: 0.2009 - accuracy: 0.9185 - 93s/epoch - 198ms/step
Epoch 3/10
469/469 - 91s - loss: 0.1683 - accuracy: 0.9329 - 91s/epoch - 194ms/step
Epoch 4/10
469/469 - 90s - loss: 0.1448 - accuracy: 0.9429 - 90s/epoch - 191ms/step
Epoch 5/10
469/469 - 89s - loss: 0.1271 - accuracy: 0.9498 - 89s/epoch - 191ms/step
Epoch 6/10
469/469 - 91s - loss: 0.1181 - accuracy: 0.9524 - 91s/epoch - 194ms/step
Epoch 7/10
469/469 - 91s - loss: 0.1063 - accuracy: 0.9569 - 91s/epoch - 193ms/step
Epoch 8/10
469/469 - 92s - loss: 0.0934 - accuracy: 0.9639 - 92s/epoch - 196ms/step
Epoch 9/10
469/469 - 92s - loss: 0.0874 - accuracy: 0.9662 - 92s/epoch - 197ms/step
Epoch 10/10
469/469 - 91s - loss: 0.0786 - accuracy: 0.9690 - 91s/epoch - 195ms/step


In [ ]:
# Arreglar este número
validation_size = 3500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]

In [ ]:
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Acc: %.2f" % (acc))

122/122 - 3s - loss: 0.2504 - accuracy: 0.9271 - 3s/epoch - 27ms/step
Score: 0.25
Acc: 0.93


In [ ]:
import numpy as np

In [ ]:
# Esto no funciona cuando pongo las tres etiquetas, cuando son dos si funciona.
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1

print("pos_acc", pos_correct/pos_cnt*100, "%")
#print("neu_acc", neu_correct/neu_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

1/1 - 0s - 346ms/epoch - 346ms/step
1/1 - 0s - 35ms/epoch - 35ms/step
1/1 - 0s - 30ms/epoch - 30ms/step
1/1 - 0s - 37ms/epoch - 37ms/step
1/1 - 0s - 29ms/epoch - 29ms/step
1/1 - 0s - 28ms/epoch - 28ms/step
1/1 - 0s - 31ms/epoch - 31ms/step
1/1 - 0s - 27ms/epoch - 27ms/step
1/1 - 0s - 28ms/epoch - 28ms/step
1/1 - 0s - 30ms/epoch - 30ms/step
1/1 - 0s - 28ms/epoch - 28ms/step
1/1 - 0s - 29ms/epoch - 29ms/step
1/1 - 0s - 29ms/epoch - 29ms/step
1/1 - 0s - 27ms/epoch - 27ms/step
1/1 - 0s - 27ms/epoch - 27ms/step
1/1 - 0s - 28ms/epoch - 28ms/step
1/1 - 0s - 25ms/epoch - 25ms/step
1/1 - 0s - 25ms/epoch - 25ms/step
1/1 - 0s - 32ms/epoch - 32ms/step
1/1 - 0s - 33ms/epoch - 33ms/step
1/1 - 0s - 38ms/epoch - 38ms/step
1/1 - 0s - 35ms/epoch - 35ms/step
1/1 - 0s - 45ms/epoch - 45ms/step
1/1 - 0s - 35ms/epoch - 35ms/step
1/1 - 0s - 35ms/epoch - 35ms/step
1/1 - 0s - 43ms/epoch - 43ms/step
1/1 - 0s - 37ms/epoch - 37ms/step
1/1 - 0s - 37ms/epoch - 37ms/step
1/1 - 0s - 39ms/epoch - 39ms/step
1/1 - 0s - 3

In [ ]:
'''Este código es para guardar la base, probablemente no lo use de vuelta.

base_tit_gral.to_csv('base_titulares_sentimiento.csv', encoding='iso8859_2', index=False) 
files.download('base_titulares_sentimiento.csv')'''

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
base_tit_gral_sentimiento = pd.read_csv('base_titulares_sentimiento.csv', header=None, encoding='iso8859_2')

In [ ]:
base_tit_gral_sentimiento.columns = ['titular', 'fecha', 'diario', 'sentimiento']
base_tit_gral_sentimiento.head(10)# Tengo que eliminar la primer fila.

,titular,fecha,diario,sentimiento
0,titular,fecha,diario,sentimiento
1,"""Mirá la ceremonia de inauguración de los Jueg...","""2021-07-23""",elobse,NEU
2,"""El contrasentido del clásico: Peńarol perdió ...","""2021-07-23""",elobse,NEU
3,"""Los sueldos de $ 200.000 en el Directorio de ...","""2021-07-23""",elobse,NEU
4,"""żDónde se podrán ver los Juegos Olímpicos de ...","""2021-07-23""",elobse,NEU
5,"""Crece interés de argentinos por Montevideo: c...","""2021-07-23""",elobse,POS
6,"""Delgado y Orsi, la tensión entre dos referent...","""2021-07-23""",elobse,NEU
7,"""El antiguo PES será gratuito: żhay chances de...","""2021-07-23""",elobse,NEU
8,"""El correctivo salarial que se debe pagar a tr...","""2021-07-23""",elobse,NEU
9,"""\""Obras que no se harán\"", \""sorpresa\"" y \""f...","""2021-07-23""",elobse,NEG
